In [79]:
# Importing the Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas_datareader import data as pdr
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

import yfinance as yf

yf.pdr_override()

In [12]:
#Defining data
crypto_curr = 'BTC'
actual_curr = 'USD'

start = dt.datetime(2019,1,1)
end = dt.datetime.now()

# Defining Training Set
training_set = pdr.get_data_yahoo(f'{crypto_curr}-{actual_curr}', start = start, end = end)

print("Overview of training set: \n",training_set.head())
print("\nSize of Training set:",len(training_set))

[*********************100%***********************]  1 of 1 completed
Overview of training set: 
                                   Open         High          Low        Close  \
Date                                                                            
2019-01-01 00:00:00+00:00  3746.713379  3850.913818  3707.231201  3843.520020   
2019-01-02 00:00:00+00:00  3849.216309  3947.981201  3817.409424  3943.409424   
2019-01-03 00:00:00+00:00  3931.048584  3935.685059  3826.222900  3836.741211   
2019-01-04 00:00:00+00:00  3832.040039  3865.934570  3783.853760  3857.717529   
2019-01-05 00:00:00+00:00  3851.973877  3904.903076  3836.900146  3845.194580   

                             Adj Close      Volume  
Date                                                
2019-01-01 00:00:00+00:00  3843.520020  4324200990  
2019-01-02 00:00:00+00:00  3943.409424  5244856836  
2019-01-03 00:00:00+00:00  3836.741211  4530215219  
2019-01-04 00:00:00+00:00  3857.717529  4847965467  
2019-01-05 00:00:

In [21]:
# Normalization of training set
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(training_set['Close'].values.reshape(-1,1))
print("The training_set[Close] normalised between 0-1:\n",scaled_data)

The training_set[Close] normalised between 0-1:
 [[0.00692016]
 [0.00847686]
 [0.00681452]
 ...
 [0.29331632]
 [0.32584123]
 [0.33091703]]


In [25]:
# Length of the Data set
training_set_size = len(scaled_data) - 1

X_train = scaled_data[0:training_set_size]
Y_train = scaled_data[1:training_set_size + 1]

X_train

array([[0.00692016],
       [0.00847686],
       [0.00681452],
       ...,
       [0.28688465],
       [0.29331632],
       [0.32584123]])

In [27]:
#Creating neural network
model = Sequential()

model.add(LSTM(units=50, return_sequences = True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))    
model.add(Dense(units=1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(X_train, Y_train, batch_size = 32, epochs = 100)

Epoch 1/100
48/48 [==============================] - 5s 3ms/step - loss: 0.1093
Epoch 2/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0299
Epoch 3/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0051
Epoch 4/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0043
Epoch 5/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0038
Epoch 6/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0035
Epoch 7/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0031
Epoch 8/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0028
Epoch 9/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0028
Epoch 10/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0023
Epoch 11/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0025
Epoch 12/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0023
Epoch 13/100
48/48 [=================

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             10400     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 50)             20200     
                                                                 
 dropout_1 (Dropout)         (None, 1, 50)             0         
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                

In [30]:
# Testing data
test_start = dt.datetime(2020,1,1)
test_end = dt.datetime.now()
test_set = pdr.get_data_yahoo(f'{crypto_curr}-{actual_curr}', start = test_start, end = test_end)
test_set.head()
print("Overview of test set: \n",test_set.head())
print("\nSize of Test set:",len(test_set))

[*********************100%***********************]  1 of 1 completed
Overview of training set: 
                                   Open         High          Low        Close  \
Date                                                                            
2020-01-01 00:00:00+00:00  7194.892090  7254.330566  7174.944336  7200.174316   
2020-01-02 00:00:00+00:00  7202.551270  7212.155273  6935.270020  6985.470215   
2020-01-03 00:00:00+00:00  6984.428711  7413.715332  6914.996094  7344.884277   
2020-01-04 00:00:00+00:00  7345.375488  7427.385742  7309.514160  7410.656738   
2020-01-05 00:00:00+00:00  7410.451660  7544.497070  7400.535645  7411.317383   

                             Adj Close       Volume  
Date                                                 
2020-01-01 00:00:00+00:00  7200.174316  18565664997  
2020-01-02 00:00:00+00:00  6985.470215  20802083465  
2020-01-03 00:00:00+00:00  7344.884277  28111481032  
2020-01-04 00:00:00+00:00  7410.656738  18444271275  
2020-01-05 

In [54]:
actual_prices = test_set['Close'].values
print("Actual BTC Prices :\n", actual_prices)

model_inputs = actual_prices.reshape(-1,1)
model_inputs = scaler.transform(model_inputs)
print("\nModel Inputs :\n",model_inputs)

model_inputs = np.array(model_inputs)
model_inputs = np.reshape(model_inputs, (model_inputs.shape[0], 1, model_inputs.shape[1]))
prediction_prices = model.predict(model_inputs)
prediction_prices = scaler.inverse_transform(prediction_prices)
print("\nPredicted prices :\n",prediction_prices)

Actual BTC Prices :
 [ 7200.17431641  6985.47021484  7344.88427734 ... 22220.8046875
 24307.84179688 24619.8671875 ]

Model Inputs :
 [[0.05923109]
 [0.05588509]
 [0.06148629]
 ...
 [0.29331632]
 [0.32584123]
 [0.33070391]]
36/36 [==============================] - 0s 1ms/step

Predicted prices :
 [[ 8095.108 ]
 [ 7904.8037]
 [ 8223.668 ]
 ...
 [22572.924 ]
 [24731.396 ]
 [25056.209 ]]


In [71]:
import plotly.graph_objects as go

dates = pd.date_range(start='2020-01-01', end=pd.Timestamp.now(), freq='D')
date_array = dates.to_numpy()

prediction_price = prediction_prices.flatten()

# Garph

x1 = dates
y1 = actual_prices
x2 = dates
y2 = prediction_price

f1 = go.Figure(
    data = [
        go.Scatter(x=x1, y=y1, name="Actual Prices"),
        go.Scatter(x=x2, y=y2, name="Predicted Prices"),
    ],
    layout = {"xaxis": {"title": "Date"}, "yaxis": {"title": "Price"}, "title": "Price Prediction"}
)
f1


[ 8095.108   7904.8037  8223.668  ... 22572.924  24731.396  25056.209 ]


In [81]:
# Predicting the next day
last_price = actual_prices[-1]

last_price_scaled = scaler.transform(np.array([[last_price]]))
last_price_scaled_reshaped = np.reshape(last_price_scaled, (1, 1, 1))

predicted_price_scaled = model.predict(last_price_scaled_reshaped)
predicted_price = scaler.inverse_transform(predicted_price_scaled)


prediction_date = dt.date.today() + dt.timedelta(days=1)
print(f"Predicted Price for {prediction_date} :",predicted_price[0][0])

1/1 [==============================] - 0s 26ms/step
Predicted Price for 2023-02-19 : 25056.209
